# Day13  
本章以day11中的CNNs为例，测试`tensorboard`监控、`torch.save()`保存、`onnx`导出。

![tensorboard](../images/tensorboard.png)

代码在day13.py中查看。

先在终端运行：
```
pip install tensorboard
```

在终端运行：  
```
cd handonML-
tensorboard --logdir=runs
```
路径是相对于你当前终端所在目录的。如果你在 handonML- 目录下运行，路径就是 runs。   
打开弹出的网页（如localhost:6006），这时候里面是空的。   
执行训练脚本。
```
python3 scripts/DL/day13.py
```

`tensorboard`总结表格：
| 功能模块      | 主要作用                                 | 常用 PyTorch API 方法 (SummaryWriter 的方法) | 适用场景/意义                                               |
|--------------|------------------------------------------|---------------------------------------------|------------------------------------------------------------|
| Scalars      | 监控数值指标随时间变化（曲线图）           | add_scalar()                                | 训练损失、验证损失、准确率、学习率等核心指标监控            |
| Graphs       | 可视化模型网络结构和数据流                 | add_graph()                                 | 理解复杂模型结构，Debug 数据流问题                          |
| Histograms   | 查看参数和梯度随时间的分布变化             | add_histogram()                             | 诊断梯度问题（消失/爆炸），观察参数学习过程                 |
| Images       | 可视化图像数据                             | add_images()                                | 检查数据预处理，查看模型输出图像，可视化中间层特征/权重      |
| Text         | 可视化文本数据                             | add_text()                                  | 查看输入/输出文本，注意力权重等字符串信息                   |
| Projector    | 可视化高维嵌入向量（降维到 2D/3D）         | add_embedding()                             | 理解嵌入空间的结构，观察相似样本的聚类情况                  |
| PR Curves    | 可视化分类模型的 Precision-Recall 曲线      | add_pr_curve()                              | 评估分类模型性能，特别是不平衡数据，关注不同阈值下的权衡     |
| (Comparison) | 对比多个实验的指标曲线                     | (通过 logdir 目录结构实现)                   | 选择最佳超参数、模型结构，进行 A/B 测试                     |


`torch.save()`保存模型

保存模型（不推荐）


In [ ]:
"""
torch.save(model,pth)
...
loaded_model = torch.load(pth)
"""

保存参数（推荐）

In [ ]:
"""
torch.save(model.state_dict(), pth)
loaded_model = MyModel()
loaded_model.load_state_dict(torch.load(pth))
"""

保存checkpoint（加载优化器状态继续训练）

In [1]:
"""
checkpoint = {
    'epoch': epoch,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss': loss,
}
torch.save(checkpoint, pth)

...

model = MyModel()
optimizer = MyOptimizer()
scheduler = MyScheduler()

checkpoint = torch.load(pth)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
start_epoch = checkpoint['epoch']
loss = checkpoint['loss']
"""

"\ncheckpoint = {\n    'epoch': epoch,\n    'model_state_dict': model.state_dict(),\n    'optimizer_state_dict': optimizer.state_dict(),\n    'loss': loss,\n}\ntorch.save(checkpoint, pth)\n...\ncheckpoint = torch.load(pth)\nmodel = MyModel()\nmodel.load_state_dict(checkpoint['model_state_dict'])\noptimizer = MyOptimizer()\noptimizer.load_state_dict(checkpoint['optimizer_state_dict'])\nscheduler = MyScheduler()\nscheduler.load_state_dict(checkpoint['scheduler_state_dict'])\nstart_epoch = checkpoint['epoch']\nloss = checkpoint['loss']\n"

可以选择加载已保存的checkpoint继续训练，见model/SimpleCNN/checkpoint_{epoch}.pth。保存的都是模型训练至当前的最优状态，因此最大的保存epoch数是全局最优状态。为了节省空间，我把全局最优状态保存，其他checkpoint都删除。   

终端输入：   
`cd handonML-/scripts/DL/`    
`python3 day13.py -h` # 查看帮助    
`python3 day13.py --checkpoint x` # 输入epoch版本    
`python3 day13.py --checkpoint 86 ` #例如：加载并评估第86个epoch的checkpoint（只预测）



值得注意的是，在我反复训练过程中，发现模型在验证集上的val_loss普遍小于在训练集上的train_loss.这是由于：   

- __Dropout:__ 在训练阶段，Dropout 会随机关闭一部分神经元，这增加了训练过程的噪声，使得模型在训练集上的表现（loss）相对较差。但在验证/测试阶段 (`model.eval()`)，Dropout 是关闭的，所有神经元都参与计算，这使得模型能够利用其全部的学习能力，通常会在验证集上获得更好的表现，表现为 loss 较低。

- __Batch Normalization:__ Batch Normalization 在训练和验证阶段的行为也不同。训练时，它使用当前 mini-batch 的均值和方差来归一化数据。验证时，它使用在整个训练集上计算得到的（或训练过程中累积的）全局均值和方差。这种差异也会导致训练 loss 和验证 loss 之间的差异。

- __数据增强 (Data Augmentation):__ 你使用了 `RandomHorizontalFlip` 和 `RandomCrop` 等数据增强技术。这些技术只在训练阶段应用，增加了训练数据的多样性和难度，从而可能提高训练 loss。在验证阶段，数据增强通常是关闭的，验证集数据是原始的，这使得模型在验证集上更容易获得较低的 loss。


但是似乎模型在近50个epoch之后就收敛（到局部最优），泛化能力不再增加了。原因可能有：   
- 模型本身太简单
- 学习率衰减速度过快（StepLR）

由于已经达到70%的目标，本阶段不再继续优化。

`onnx`

ONNX 是一个开放的生态系统，旨在让不同的深度学习框架（如 PyTorch, TensorFlow, Keras, MXNet, PaddlePaddle 等）之间能够互相转换模型！

ONNX 的核心概念：  
- ONNX Graph (计算图): ONNX 文件内部存储的是模型的计算图，描述了数据流经模型时的各种操作（加法、乘法、卷积、ReLU 等）以及它们之间的连接关系。  
- ONNX Operators (算子): ONNX 定义了一套标准的算子库，所有的操作都必须是 ONNX 支持的算子。这是实现跨框架兼容的基础。  
- ONNX Protobuf 格式: ONNX 模型使用 Protocol Buffers 格式进行序列化，这是一种高效、语言中立的数据交换格式。